# Single Cell RGC Subtype Marker Expression Analysis in Day 21 RGC-iNs

In [ ]:
import scanpy as sc
import matplotlib.pyplot as plt

sc.settings.verbosity = 3 # hints 

### Data Collection

In [ ]:
# Read data into an AnnData object (this step may take a few minutes the first time)
adata = sc.read_10x_mtx(
    './../../sc_data/d21/',
    var_names='gene_symbols',
    cache=True
)

adata.var_names_make_unique()
display(adata)

### Preprocessing

In [ ]:
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

# annotate the group of mitochondrial genes as 'mt' for later removal
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'

# generate plots to assess cell/transcript quality
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

In [ ]:
# filter out cells with an abnormal number of total_counts and those with high mitochondrial gene presence
adata = adata[adata.obs.total_counts < 150000, :]
adata = adata[adata.obs.pct_counts_mt < 5, :]

# normalize and logarthmize data
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

display(adata)

### Clustering and Dimensionality Reduction

In [ ]:
sc.tl.pca(adata, svd_solver='arpack', use_highly_variable=False)
sc.pl.pca(adata)
sc.pl.pca_variance_ratio(adata, log=True)

In [ ]:
sc.pp.neighbors(adata)
sc.tl.louvain(adata)
sc.tl.umap(adata)

### UMAP Visualization

In [ ]:
sc.pl.umap(adata, color=['louvain'], legend_loc='on data', save='d21_louvain.pdf')

### Subtype Marker Expression Analysis

In [ ]:
# Assign alpha, ds, and ip RGC subtype markers
subtypes = {
    'alpha':['SPP1', 'PIGO', 'RPP14', 'PANK1', 'CALB2', 'GAL', 'CRH'],
    'DS':['DCX', 'CDH6', 'JAM2', 'FSTL4', 'GSN', 'LGI1', 'MMP17', 'COL25A1'],
    'ip':['OPN4']
}

sc.pl.dotplot(adata,subtypes, groupby='louvain',
                vmax=1, mean_only_expressed=True, dot_max=1, swap_axes=True,
                title='RGC Subtype Markers',
                save='d21_subtype_dotplot.pdf'
)

In [ ]:
# Assign Sucseptible/Resiliant RGC subtype markers
sus_res = {
    'Susceptible':[
        'GABRA2',
        'CARTPT',
        'TFAP2D',
        'MGLL',
        'MYO1B',
        'CRHBP',
        'HPCAL1',
        'VSTM2B',
        'TAC1',
        'EVC2',
        'NEUROD2',
        'PENK',
        'PMP22',
        'LMO1'
    ],
    'Resilient':[
        'GLDN',
        'CHL1',
        'QPCT',
        'TBX20',
        'DGKG',
        'TIMP2',
        'IFITM10',
        'EGR1',
        'EOMES',
        'JUNB',
        'IGFBP2',
        'OSTF1',
        'FGF1',
        'SEMA5A',
        'ESRRG',
        'KBTBD11',
        'RAMP1',
        'PRKCQ',
        'CTXN3',
        'NDNF',
        'MAN1A1',
        'SDK2',
        'PRPH',
        'SDK1',
        'IGF1',
        'IFI27'
    ]
}

sc.pl.dotplot(adata, sus_res, groupby='louvain',
                vmax=1, mean_only_expressed=True, dot_max=0.4,
                title='Susceptible/Resilient Markers',
                save='d21_susceptible_resiliant.pdf',
)

### Neurotrphic Factor Receptor Expression Analysis

In [ ]:
# List of Neurotrophic Factor receptors we want to explore
nf_receptors = ['NTRK1', 'NTRK2', 'NTRK3', 'GFRA1', 'NGFR', 'CNTFR']

sc.pl.dotplot(adata, nf_receptors, groupby='louvain',
                vmax=1, mean_only_expressed=True, dot_max=0.6,
                title='Neurotrophic Receptor Markers',
                save='d21_neurotrophic_markers.pdf'
)

Written by Manan Chopra (m1chopra@ucsd.edu) @ Wahlin Lab  
Last updated on May 22, 2023